In [1]:
from neural_astar_jax.utils import data
from neural_astar_jax.utils import training
import jax
from importlib import reload
from neural_astar_jax.planner.astar import NeuralAstar
reload(data)
reload(training)
import optax
from flax.training.train_state import TrainState

In [2]:
dataloader = data.MazeDataLoader(filename="../../planning-datasets/data/mpd/alternating_gaps_032_moore_c8.npz", split="train", batch_size=100)
batch = dataloader.sample_batch(jax.random.PRNGKey(0))

In [3]:
planner = NeuralAstar(is_training=True, search_step_ratio=0.05)
variables = planner.init(jax.random.PRNGKey(0), batch.map_design, batch.start_map, batch.goal_map)

In [4]:
state = TrainState.create(apply_fn=jax.jit(planner.apply), params=variables, tx=optax.adam(learning_rate=0.001))


In [6]:
%time training.step(batch, state, True)

CPU times: user 13.4 s, sys: 1.48 s, total: 14.9 s
Wall time: 3.01 s


(DeviceArray(0.03090332, dtype=float32),
 TrainState(step=DeviceArray(1, dtype=int32, weak_type=True), apply_fn=<CompiledFunction of <bound method Module.apply of NeuralAstar()>>, params=FrozenDict({
     params: {
         encoder: {
             Conv_0: {
                 bias: DeviceArray([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                              0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                              0., 0.], dtype=float32),
                 kernel: DeviceArray([[[[-1.59661189e-01,  1.40554547e-01,  3.90278488e-01,
                                 -2.50529647e-01,  1.16090858e-02,  1.03038557e-01,
                                 -2.11880371e-01, -9.17382762e-02,  2.37546518e-01,
                                  5.35270944e-02, -5.32692038e-02, -3.12981635e-01,
                                 -9.77745950e-02,  2.35913530e-01,  1.79319501e-01,
                                 -1.00310005e-01, -4.64828998e-01, -5.642